In [177]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [178]:
credit = pd.read_csv('Z:\\ML\\Movie_recommender_sys\\dataset\\tmdb_5000_credits.csv')
movies = pd.read_csv('Z:\\ML\\Movie_recommender_sys\\dataset\\tmdb_5000_movies.csv')

## Data Preprocessing

In [179]:
movies = movies.merge(credit,on='title')

In [180]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [181]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [182]:
# use necessary column insted of dropping column
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [183]:
# check number of rows and column
movies.shape

(4809, 7)

In [184]:
# checked weather dataset have null values
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [185]:
# only thre values are misssing that's why we removed
movies.dropna(inplace=True)

In [186]:
# check duplicated rows
movies.duplicated().sum()

0

In [187]:
movies.iloc[0].genres
# movies['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [188]:
# preprocess the genres column
import ast
def convert(obj):
    list = []
    for i in ast.literal_eval(obj): # use literal_eval to convert string of list to list
        list.append(i['name'])
    return list


In [189]:
# apply on return list
movies['genres'] = movies['genres'].apply(convert)

In [190]:
# apply function
movies['keywords'] = movies['keywords'].apply(convert)

In [191]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [192]:
def cast_mine(obj):
    list1 = []
    count = 0
    for i in ast.literal_eval(obj):
        list1.append(i['name'])
        count += 1
        if count == 3:
            break
    return list1

movies['cast'] = movies['cast'].apply(cast_mine)

In [193]:
# get director name from the crew column
def fetch_director(obj):
    list2 = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            list2.append(i['name'])
            break
    return list2

movies['crew'] = movies['crew'].apply(fetch_director)

In [194]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [195]:
# overview

movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [196]:
# some first_names are commin to another crew it creates problem thast why we used replace function

movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [197]:
# concatenate genres, keywords, cast, crew column in new column called tags

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [198]:
df = movies[['movie_id','title','tags']]
df.head(2)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."


In [199]:
# convert list into string for tags column

df['tags'] = df['tags'].apply(lambda x: " ".join(x))

In [200]:
df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [201]:
# convert into lower becasue of further use, easyness and case-sensitive.

df['tags'] = df['tags'].apply(lambda x: x.lower())


In [202]:
# most of the words are common convert into single entity
# like: run, running, runed into run 
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [203]:
def stem(text):
    list = []
    for i in text.split():
        list.append(ps.stem(i))
    return " ".join(list)

df['tags'] = df['tags'].apply(stem)

0       in the 22nd century, a parapleg marin is dispa...
1       captain barbossa, long believ to be dead, ha c...
2       a cryptic messag from bond’ past send him on a...
3       follow the death of district attorney harvey d...
4       john carter is a war-weary, former militari ca...
                              ...                        
4804    el mariachi just want to play hi guitar and ca...
4805    a newlyw couple' honeymoon is upend by the arr...
4806    "signed, sealed, delivered" introduc a dedic q...
4807    when ambiti new york attorney sam is sent to s...
4808    ever sinc the second grade when he first saw h...
Name: tags, Length: 4806, dtype: object

## Vectorization

In [208]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [209]:
vectors = cv.fit_transform(df['tags']).toarray() # convert into vector like co-ordinate from x-axis and y-axis for each movie
# to measure the distance between the movie
# we dont use euclidean distance it fails on higher data
# instead we use cosine like measure the angle between lines


In [210]:
cv.get_feature_names_out().tolist() # convert array to list

['000',
 '007',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '17th',
 '18',
 '18th',
 '18thcenturi',
 '19',
 '1910',
 '1920',
 '1930',
 '1940',
 '1944',
 '1950',
 '1950s',
 '1960',
 '1960s',
 '1970',
 '1970s',
 '1971',
 '1974',
 '1976',
 '1980',
 '1985',
 '1990',
 '1999',
 '19th',
 '19thcenturi',
 '20',
 '200',
 '2003',
 '2009',
 '20th',
 '21st',
 '23',
 '24',
 '25',
 '30',
 '300',
 '3d',
 '40',
 '50',
 '500',
 '60',
 '70',
 '80',
 'aaron',
 'aaroneckhart',
 'abandon',
 'abduct',
 'abigailbreslin',
 'abil',
 'abl',
 'aboard',
 'abov',
 'abus',
 'academ',
 'academi',
 'accept',
 'access',
 'accid',
 'accident',
 'acclaim',
 'accompani',
 'accomplish',
 'account',
 'accus',
 'ace',
 'achiev',
 'acquaint',
 'act',
 'action',
 'actionhero',
 'activ',
 'activist',
 'activities',
 'actor',
 'actress',
 'actual',
 'ad',
 'adam',
 'adamsandl',
 'adamshankman',
 'adapt',
 'add',
 'addict',
 'adjust',
 'admir',
 'admit',
 'adolesc',
 'adopt',
 'ador',
 'adrienbrodi',
 'adult'

In [212]:
# to measure the angle we have libraries called cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors) # 4806 * 4806

In [214]:
similarity[0] # angles between movie

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

In [226]:
# function gives recommended movies
def recommend(movie):
    movie_index = df[df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list =  sorted(list(enumerate(distances)),reverse=True,key=lambda x: x[1])[1:6]
    
    for i in movie_list:
        print(df.iloc[i[0]].title)

In [235]:
movie_name = input()
recommend(movie_name)

 Batman


Batman
Batman & Robin
Batman Begins
Batman Returns
The R.M.


In [232]:
import pickle
pickle.dump(df,open('movies.pkl','wb'))

In [ ]:
pickle.dump(df,open('similarity.pkl','wb'))